In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

In [3]:
all_waves = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/df_combined.csv', low_memory=False)

For this preliminary analysis, I will only use wave 2,4,6. \
Each wave's data was collected on 2020-06-17 - 2020-06-23, 2020-11-29 - 2020-12-16, 2021-05-12 - 2021-05-25.

In [4]:
all_waves['wave_range'].unique()

array(['2020-06-17 - 2020-06-23', '2020-11-29 - 2020-12-16',
       '2021-05-12 - 2021-05-25'], dtype=object)

In [5]:
col_types = {
    'ZIP': str,
    'CD':str,
    'COUNTY_FIPS':str,
    'STATE_FIPS':str
}

ZIP_FEATURES = pd.read_csv('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/data/ZIP_Features.csv',
                          dtype=col_types)

In [6]:
col_types = {
    'resp_zip': str
}

all_waves = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/df_combined.csv', 
                        low_memory=False, dtype=col_types)

all_waves = all_waves.rename(columns={
    'resp_zip': 'ZIP'
})

columns_to_keep = ['ResponseId','StartDate','resp_yob','resp_sex','resp_hispanic','resp_race_1','resp_race_2',
                  'resp_race_3','resp_race_4','resp_race_5','resp_race_6','resp_educ','resp_nativity','ZIP',
                  'resp_hhsize','resp_employ','resp_occupation','lefthome_num','num_cc_nonhh',
                  'lefthome_where_1','lefthome_where_2','lefthome_where_3','lefthome_where_4','lefthome_where_10',
                  'lefthome_where_8','lefthome_where_9','lefthome_where_5','lefthome_where_11','lefthome_where_6',
                  'lefthome_where_7','inet_freq','socmedia_use','covid19_familiar','covid19_concern',
                  'covid19_f2fchange','covid19_reduceOK','policy_sip','age','hhi','political_party','political_view',
                  'industry','health_insurance','interview_date','wave','agecat','city','covid19_vax','covid19_whynot_vax']
all_waves = all_waves[columns_to_keep]

In [7]:
BICS_ZIP_FEATURES = pd.merge(all_waves, ZIP_FEATURES, on = "ZIP", how = "left")
BICS_ZIP_FEATURES.to_csv('data/BICS_ZIP_Features.csv',index=False)

In [ ]:
BICS_ZIP_FEATURES